# Building a custom task and network

The purpose of this notebook is to create a network to control a plant, and to declare a task that the network can learn through the optimization process. The actual optimization process, and how to save and re-load a network will be discussed in another notebook.

For how to build a plant from scratch, feel free to look up the `1-build-plant.ipynb` notebook.

Let's start by importing what we need.


In [1]:

import os
import sys
import numpy as np
import tensorflow as tf

colab_env = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
colab_initialized = True if os.path.exists("MotorNet") else False

if colab_env and not colab_initialized:
  !git clone https://github.com/OlivierCodol/MotorNet
  sys.path.append('MotorNet')
  print("Running cell using COLAB initialization...")
elif colab_env and colab_initialized:
  print("Already initialized using COLAB initialization.")
else:
  paths = [p for p in sys.path if os.path.exists(p)]
  local_initialized = True if [p for p in paths if "motornet" in os.listdir(p)] else False
  if local_initialized:
    %load_ext autoreload
    %autoreload 2
    print("Already initialized using LOCAL initialization.")
  else:
    path = [p for p in paths if p.__contains__("tutorials")]
    if len(path) != 1:
      raise ValueError("Path to MotorNet could not be determined with certainty.")
    sys.path.append(os.path.dirname(path[:path.rfind('tutorials')]))
    %load_ext autoreload
    %autoreload 2
    print("Running cell using LOCAL initialization...")


import motornet as mn

print('All packages imported.')
print('tensorflow version: ' + tf.__version__)
print('numpy version: ' + np.__version__)


Already initialized using LOCAL initialization.
All packages imported.
tensorflow version: 2.9.1
numpy version: 1.22.4



# I. Introduction

Since the purpose of this notebook is not to show how to build a plant, we will use a pre-built plant that comes with the `motornet` toolbox. This is a 4-muscles point mass plant, with `ReluMuscle` actuators.

We will also use the default `motornet` network, which is a GRU network. Since we only specify an integer (rather than a list of integers) for `n_units`, this will end up being a one-layer GRU network.

Generally speaking, the objects we create follow the hierarchical structure illustrated below.





<img src="img/hierarchy.png" alt="drawing" width="500"/>


In [2]:

plant = mn.plants.ReluPointMass24()
network = mn.nets.layers.GRUNetwork(plant=plant, n_units=50, kernel_regularizer=10**-6, name='network')


# II. Subclassing a Task object

Let us go over a simple subclassing process to create a custom task.


### II. 1. Initialization of the Task subclass
The base class for tasks is `mn.tasks.Task`, so we will make the custom task inherit from that base class. We can then define a `__name__` for the class, and declare the losses in the `__init__` method using a `self.add_loss` method that is inherited from the base class.

Note that declaring losses is not mandatory for the simulation to run, but this is easier, as otherwise we would have to manually do all the things that `add_loss` does automatically for us. Also this may result in misleading (shuffled) loss labels when printing progress bars at runtime because `tensorflow` models (`tf.keras.Model`) do not maintain loss label orders properly for some reason. Adding losses to the task object will make it available to our curstom-made `tf.keras.Model` subclass, which is `mn.nets.MotorNetModel`. In `MotorNetModel` the losses available in `Task` subclasses will be re-ordered properly to avoid the parent class `tf.keras.Model` reshuffling the labels in a wrong order.

The `add_loss` method takes three arguments:
- `assigned_output` should be a string matching one of the keys of the `MotorNetModel`'s output dictionary. Of note, each key corresponds to a state of the model. In its current version, `motornet` only allows one loss per output key.
- `loss` should pass a loss object, either custom-made or from the `mn.nets.losses` module.
- `loss_weight` is a scalar that defines the weight of the loss in the global loss that the network will optimize at runtime.


### II. 2. Generating inputs
We can next define the `task.generate` method. It should take as inputs:
- `batch_size`
- `n_timesteps`

If desired, it can also take `**kwargs` inputs.

It should produce as output a list containing three items, in the order below:
- A dictionary containing the inputs to the network.
- The targets that will be passed to the network (the `y`, or `y_true` in the nomenclature of the TensorFlow documentation).
- The initial states to the network.


#### II. 2. a. Input dictionary
The input dictionary only requires a "inputs" key. The value assigned to that key will be passed as-is as input to the network for a forward pass. So is content is essentially up to the user, according to what the user wishes the network to receive as input information. Typically, for a reaching movement, this could be the target's position in cartesian coordinates. If a delayed reach is desired, one could consider adding a go cue as well.

Of note, the first dimension (rows) should always be  `batch_size` and the second dimension (columns) should always be the number of timesteps for the reach. This is true for any value in that dictionary, not just for the value associated with the "inputs" key.

Other notable keys one may add are "joint_load" and "endpoint_load". These would be transmitted to the plant by the network. If one wishes to add more keys to the input dictionary, it would be required to subclass the network to implement appropriate handling of these custom-made keys in the `call` method of the network.

The flexibility from using a dictionary as input is the reason we decided to use them. Note however that generally, `tensorflow` models can accept other forms of inputs, such as simple arrays, though our `motornet` model does not.


#### II. 2. b. Targets
The target values that the plant should produce, and that will be passed to the loss functions. In `tensorflow` nomenclature, this is sometimes referred to as the `y_true`, to which the `y_pred` is compared.
In practice this could be the position of the target for a reaching movement. If a delayed movement is desired, it could be the starting position until the go cue time, and the target position afterwards.


#### II. 2. c. Initial states
The value of the initial states at simulation start. These can be obtained from the `task.get_initial_state(batch_size=batch_size)` method. If a pre-defined starting position is desired, one can pass an optional `joint_state` argument to this method as well.




In [3]:


class RandomTargetReachWithLoads(mn.tasks.Task):
    def __init__(self, network, endpoint_load: float, **kwargs):
        super().__init__(network, **kwargs)
        self.__name__ = 'RandomTargetReachWithLoads'
        self.endpoint_load = endpoint_load

        # losses
        max_iso_force = self.network.plant.muscle.max_iso_force
        c_loss = mn.nets.losses.PositionLoss()
        m_loss = mn.nets.losses.L2ActivationLoss(max_iso_force=max_iso_force)
        self.add_loss(assigned_output='cartesian position', loss=c_loss, loss_weight=1.)
        self.add_loss(assigned_output='muscle state', loss=m_loss, loss_weight=.2)

    def generate(self, batch_size, n_timesteps, **kwargs):
        validation = kwargs.get("validation", False)
        if not validation:
            init_states = self.get_initial_state(batch_size=batch_size)
        else:
            joint_state = tf.zeros((batch_size, self.network.plant.space_dim))
            init_states = self.get_initial_state(batch_size=batch_size, joint_state=joint_state)
        goal_states_j = self.network.plant.draw_random_uniform_states(batch_size=batch_size)
        goal_states = self.network.plant.joint2cartesian(goal_states_j)
        targets = self.network.plant.state2target(state=goal_states, n_timesteps=n_timesteps).numpy()
        endpoint_load = tf.constant(self.endpoint_load, shape=(batch_size, n_timesteps, 2))
        inputs = {"inputs": targets[:, :, :self.network.plant.space_dim], "endpoint_load": endpoint_load}
        return [inputs, targets, init_states]


task = RandomTargetReachWithLoads(network=network, endpoint_load=3.)
print("Task subclass built.\n")



L = task.generate(batch_size=5, n_timesteps=10, validation=True)

print("Dictionary content:")
for k, v in L[0].items():
    print("\t" + k + " shape :--> ", v.shape)

print("\ntargets shape :--> ", L[1].shape, "\n")

print("initial states shape:")
for elem in L[2]:
    print("\t", elem.shape)


Task subclass built.

Dictionary content:
	inputs shape :-->  (5, 10, 2)
	endpoint_load shape :-->  (5, 10, 2)

targets shape :-->  (5, 10, 4) 

initial states shape:
	 (5, 4)
	 (5, 4)
	 (5, 4, 4)
	 (5, 4, 4)
	 (5, 8, 1)
	 (5, 2, 1)
	 (5, 4)
	 (5, 50)



In `tensorflow`, creating a model requires declaring `tf.keras.layers.Input` layers that will handle the input and initial state data. Creating these `Input` layers mainly require to declare the shape of the arrays that will be passed through. The `Task` class should provide those using the `get_input_dict_layers` and `get_initial_state_layers` methods.



In [4]:

inputs = task.get_input_dict_layers()
for k, v in inputs.items():
    print(k + " :--> ", v)


inputs :-->  KerasTensor(type_spec=TensorSpec(shape=(None, None, 2), dtype=tf.float32, name='inputs'), name='inputs', description="created by layer 'inputs'")
endpoint_load :-->  KerasTensor(type_spec=TensorSpec(shape=(None, None, 2), dtype=tf.float32, name='endpoint_load'), name='endpoint_load', description="created by layer 'endpoint_load'")


In [5]:

state0 = task.get_initial_state_layers()

for s in state0:
    print(s)


KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='joint0'), name='joint0', description="created by layer 'joint0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='cartesian0'), name='cartesian0', description="created by layer 'cartesian0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4), dtype=tf.float32, name='muscle0'), name='muscle0', description="created by layer 'muscle0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4), dtype=tf.float32, name='geometry0'), name='geometry0', description="created by layer 'geometry0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 8, 1), dtype=tf.float32, name='proprio_feedback0'), name='proprio_feedback0', description="created by layer 'proprio_feedback0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2, 1), dtype=tf.float32, name='visual_feedback0'), name='visual_feedback0', description="created by layer 'visual_feedback0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.fl

# III. Building the network

In `tensorflow`, recurrent neural networks need to be wrapped around a `tf.keras.layers.RNN` layer (see [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN)). We set `return_sequences` to `True` to allow for all timesteps to be returned, rather than only the final step.

Next, we build the model using our `tf.keras.Model` subclass, `mn.nets.MotorNetModel`. The `MotorNetModel` class takes the same input as its parent class, as well as the `Task` object, which must be declared. It returns the same outputs as its parent class.

The purpose of subclassing `tf.keras.Model` is mainly to have custom saving and loading methods that are adequate to the models we created.

Finally, we can compile the model, using the losses held in the `Task` object.


In [6]:

rnn = tf.keras.layers.RNN(cell=network, return_sequences=True, name='RNN')
states_out = rnn(inputs, initial_state=state0)

model = mn.nets.MotorNetModel(inputs=[inputs, state0], outputs=states_out, name='model', task=task)
model.compile(optimizer=tf.optimizers.Adam(clipnorm=1.), loss=task.losses, loss_weights=task.loss_weights)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 endpoint_load (InputLayer)     [(None, None, 2)]    0           []                               
                                                                                                  
 inputs (InputLayer)            [(None, None, 2)]    0           []                               
                                                                                                  
 joint0 (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 cartesian0 (InputLayer)        [(None, 4)]          0           []                               
                                                                                              


Finally, we can generate inputs with a single line using the `task.generate` method, and run the model like one would any `tensorflow` model.


In [7]:

n_t = 100
n_batches = 4
batch_size = 32

[inputs, targets, init_states] = task.generate(n_timesteps=n_t, batch_size=n_batches * batch_size)
cb = model.fit(x=[inputs, init_states], y=targets, verbose=1, epochs=1, batch_size=batch_size, shuffle=False)


4/4 [==============================] - 7s 140ms/step - loss: 0.7270 - position_loss: 0.7269 - l2_activation_loss: 4.8923e-05
